In [1]:
!pip install git+https://github.com/boudinfl/pke.git
!pip install matplotlib
!python -m spacy download en_core_web_sm
!pip install contractions
!pip install autocorrect
!pip install flashtext

  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-yb49lgdz
  Running command git clone --filter=blob:none --quiet https://github.com/boudinfl/pke.git /tmp/pip-req-build-yb49lgdz
  Resolved https://github.com/boudinfl/pke.git to commit 5af1f817e0211c33ac3f90e1e86bb5c1283448e8
  Preparing metadata (setup.py) ... done
  Created wheel for pke: filename=pke-2.0.0-py3-none-any.whl size=6160276 sha256=c74310ac898779632d2b12af40e6d80145e8c1c469136d9e87d7c7f7e5df0930
  Stored in directory: /tmp/pip-ephem-wheel-cache-gn4twgmr/wheels/fa/b3/09/612ee93bf3ee4164bcd5783e742942cdfc892a86039d3e0a33
Successfully built pke
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Uninstalling typing_extensions-4.3.0:
      Successfully uninstalled typing_extensions-4.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.

# Import Libraries.

In [2]:
import re
import numpy as np
import contractions
from autocorrect import Speller 
from nltk.tokenize import sent_tokenize
from flashtext import KeywordProcessor
import nltk
nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Load & Clean dataset.

In [3]:
file = open('../input/questionge/Skill_BOK (1).txt','r',encoding ='utf-16', errors='ignore')
text =file.read()
#@title Default title text
def Clean(text):
  # Removing all the occurrences of links that ends with .com.
  Formatted_text = re.sub('\S*@\S*\s?', ' ', text)
  # Remove arabic words.
  Formatted_text = re.sub('[\u0600-\u06ff]+', ' ', Formatted_text)
  # Remove numbers.
  Formatted_text = ''.join([i for i in Formatted_text if not i.isdigit()])
  # Remove punctuation.
  Formatted_text = Formatted_text.replace("C++", "Cpp")
  punc = '''!()[]{}-;:‘’'"\|<>/?@$%^&§−*_~=<>^“”—+,'''
  for word in Formatted_text:
      if word in punc:
          Formatted_text = Formatted_text.replace(word, ' ')
  Formatted_text = Formatted_text.replace("Cpp", "C++")
  # Contraction.
  Formatted_text = contractions.fix(Formatted_text)
  # Replacing all the occurrences of \n,\\n,\t,\\ with a space.
  Formatted_text = Formatted_text.replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\\', ' ')
  # Remove duplicated (.).
  Pattern_Punct = re.compile(r'([.])\1{1,}')
  Formatted_text = Pattern_Punct.sub(r'\1', Formatted_text)
  Formatted_text = Formatted_text.replace(' . ', ' ').replace('i.e.', ' ')
  # Remove single letters.
  Formatted_text= ' '.join( [w for w in Formatted_text.split() if len(w)>1] )
  # Lowercase.
  Formatted_text = Formatted_text.lower()
  # Replacing repeatation of spaces that occur more than two times with that of one occurrence.
  Formatted_text = re.sub(' {1,}',' ', Formatted_text)
  # Correcting Mis-spelled words.
  spell = Speller(lang='en')
  Formatted_text = spell(Formatted_text)
  
  return Formatted_text

data = Clean(text)  
data

'video games which primarily target male audiences have become increasingly popular over the past decade. one study reports that the sales of video games have exceeded those of movies. as video games have become more popular and more realistic so has the need to assess the content and the impact of these images on their male audience. martins and colleagues found that video game characters were significantly larger than the average male in america. additionally they reported that video games rated for children were more likely to contain hypermuscular characters compared to video games rated for adults. harrison and bond examined exposure to video gaming magazines which are highly popular with preadolescent boys and showcase highly unrealistic muscular characters. they reported that boys in nd through th grade were more likely to desire more muscular body after exposure to video gaming magazines. collectively the results of these studies are problematic especially given the fact that c

# Keyphrases techniques.

In [4]:
import pke
# Yake.
extractor = pke.unsupervised.YAKE()
extractor.load_document(data, language='en')
extractor.candidate_selection()
extractor.candidate_weighting()
Yake = extractor.get_n_best(n=10)
# TopicRank => random walk algorithm.
extractor = pke.unsupervised.TopicRank()
extractor.load_document(data, language='en')
extractor.candidate_selection()
extractor.candidate_weighting()
Topic = extractor.get_n_best(n=10)
# PositionRank.
extractor = pke.unsupervised.PositionRank()
extractor.load_document(data, language='en')
extractor.candidate_selection()
extractor.candidate_weighting()
Position = extractor.get_n_best(n=10)
# TextRank.
extractor = pke.unsupervised.TextRank()
extractor.load_document(data, language='en')
extractor.candidate_selection()
extractor.candidate_weighting()
Text = extractor.get_n_best(n=10)
# TopicalPageRank.
extractor = pke.unsupervised.TopicalPageRank()
extractor.load_document(data, language='en')
extractor.candidate_selection()
extractor.candidate_weighting()
Topical = extractor.get_n_best(n=10)

/opt/conda/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  % sorted(inconsistent)


In [5]:
# Unique Keywords.
final_list = [*Yake, *Topic, *Topical, *Position, *Text]

counts = {}
for i, item in enumerate(final_list):
    if item[0] in counts:
        counts[item[0]] += 1
    else:
        counts[item[0]] = 1
print(counts)

{'video games': 2, 'video': 1, 'games': 1, 'playing video games': 1, 'game': 1, 'video game playing': 1, 'video game addicts': 3, 'video games rated': 1, 'playing video': 1, 'creativity': 2, 'players': 1, 'male audiences': 1, 'likely': 1, 'additional research': 1, 'xeno paulucci': 1, 'popular': 1, 'impressive learning potential': 1, 'cooked grasshopper': 1, 'female video game players': 2, 'female video game addicts': 2, 'typical video game player': 2, 'many popular video games': 2, 'video game play': 3, 'first video games pong': 2, 'online multiuser video games': 2, 'active video games': 2, 'video game activities': 2, 'video game characters': 1, 'video gaming magazines': 1, 'first video game': 1, 'original video game': 1, 'video game industry': 1, 'action video games': 1, 'casual puzzle games mobile phone games': 1, 'video games such': 1, 'higinbotham original video game': 1}


# Sentence Mapping.
For each keyword get the sentences from the summarized text containing that keyword.

In [6]:
def tokenize_sentences(text):
    sentences = [sent_tokenize(text)]
    sentences = [y for x in sentences for y in x]
    # Remove any short sentences less than 20 letters.
    sentences = [sentence.strip() for sentence in sentences if len(sentence) > 20]
    return sentences

def get_sentences_for_keyword(keywords, sentences):
    keyword_processor = KeywordProcessor()
    keyword_sentences = {}
    for word in keywords:
        keyword_sentences[word] = []
        keyword_processor.add_keyword(word)
    for sentence in sentences:
        keywords_found = keyword_processor.extract_keywords(sentence)
        for key in keywords_found:
            keyword_sentences[key].append(sentence)

    for key in keyword_sentences.keys():
        values = keyword_sentences[key]
        values = sorted(values, key=len, reverse=True)
        keyword_sentences[key] = values
    return keyword_sentences

sentences = tokenize_sentences(data)
keyword_sentence_mapping = get_sentences_for_keyword(counts, sentences)
        
print (keyword_sentence_mapping)

{'video games': ['the reasons for the lack of female participation in video games include the social stigma attached to female video game players the hostile and overbearing online behavior of some males toward female players females are not often socially rewarded for playing and the masculine imagery and game play mechanics in video games themselves games are often designed by males for other males cultural differences exist with regard to player preferences for video game activities.', 'the reasons for the lack of female participation in video games include the social stigma attached to female video game players the hostile and overbearing online behavior of some males toward female players females are not often socially rewarded for playing and the masculine imagery and game play mechanics in video games themselves games are often designed by males for other males cultural differences exist with regard to player preferences for video game activities.', 'as the creativity manifested

In [7]:
temp = []
model_phrases = dict()
# Clear duplicates.
for key, val in keyword_sentence_mapping.items():
    if val not in temp:
        temp.append(val)
        model_phrases[key] = val
# Concatenate all keyphrases sentences in model_phrases list.
model_phrases = sum(model_phrases.values(), [])
model_phrases

['the reasons for the lack of female participation in video games include the social stigma attached to female video game players the hostile and overbearing online behavior of some males toward female players females are not often socially rewarded for playing and the masculine imagery and game play mechanics in video games themselves games are often designed by males for other males cultural differences exist with regard to player preferences for video game activities.',
 'the reasons for the lack of female participation in video games include the social stigma attached to female video game players the hostile and overbearing online behavior of some males toward female players females are not often socially rewarded for playing and the masculine imagery and game play mechanics in video games themselves games are often designed by males for other males cultural differences exist with regard to player preferences for video game activities.',
 'as the creativity manifested around game i

# Haystack_Model.

In [8]:
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-nkuwm0sa/farm-haystack_51dd6c7f33f3463e9e7461a5ac75a89e
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-nkuwm0sa/farm-haystack_51dd6c7f33f3463e9e7461a5ac75a89e
  Resolved https://github.com/deepset-ai/haystack.git to commit e2e6887ee8ae6ce425f579ade589b117871372ee
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.4/381.4 kB 1.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 32.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [15]:
from haystack.nodes import QuestionGenerator, BM25Retriever, FARMReader
from haystack.document_stores import ElasticsearchDocumentStore

In [10]:
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

In [17]:
import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)
# wait until ES has started
! sleep 30

In [18]:
from haystack.pipelines import QuestionGenerationPipeline
from haystack.utils import print_questions

phrases = ''.join(model_phrases)
doc = [{"content": phrases}]


# Initialize document store and write in the documents.
document_store = ElasticsearchDocumentStore()
document_store.write_documents(doc)

# Initialize Question Generator.
question_generator = QuestionGenerator()

question_generation_pipeline = QuestionGenerationPipeline(question_generator)
for idx, document in enumerate(document_store):
    result = question_generation_pipeline.run(documents=[document])

Using sep_token, but it is not set yet.


In [19]:
# Extract the questions.
listOfDicts = result['generated_questions']
res  = {}
for line in listOfDicts:
   res.update(line)
quest = res['questions']
quest

['What are the reasons for the lack of female participation in video games?',
 'What is attached to female video game players the hostile and overbearing online behavior of some males toward female players females are not often socially rewarded for playing?',
 'What are games designed by males for?',
 'What is the social stigma attached to female video game players?',
 'What is not often socially rewarded for playing video games?',
 'What are games often designed by males for other males?',
 'What cultural differences exist with regard to player preferences for video game activities?',
 'The creativity manifested around game is considered to be specific to what?',
 'What is the conventional way of validation new assessments?',
 'What may not be the most reasonable method for creativity assessment in video games?',
 'Playing video games has become what?',
 'What has become customary and important part of everyday life for today youth?',
 'What has been exploring affordances of video ga

In [20]:
# Question Answering model.
from haystack.pipelines import ExtractiveQAPipeline
from haystack.nodes import TransformersReader
from haystack.nodes import TfidfRetriever

from haystack.document_stores import InMemoryDocumentStore

document_store2 = InMemoryDocumentStore()
document_store2.write_documents(doc)

reader = TransformersReader(model_name_or_path="distilbert-base-uncased-distilled-squad", tokenizer="distilbert-base-uncased", use_gpu=-1)
retriever = TfidfRetriever(document_store=document_store2)

pipe = ExtractiveQAPipeline(reader, retriever)

In [36]:
answers= []

for question in quest:
    prediction = pipe.run(query=question )
    for i in range(len(prediction['answers'])):
        answers.append(prediction['answers'][i].answer)

/opt/conda/lib/python3.7/site-packages/transformers/pipelines/base.py:1039: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,
/opt/conda/lib/python3.7/site-packages/transformers/pipelines/base.py:1039: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,
/opt/conda/lib/python3.7/site-packages/transformers/pipelines/base.py:1039: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,
/opt/conda/lib/python3.7/site-packages/transformers/pipelines/base.py:1039: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,
/opt/conda/lib/python3.7/site-packages/transformers/pipelines/base.py:1039: UserWarning: You seem to be using the pipelines sequentially on GPU. In 

In [53]:
# Get first answer.
def Extract(lst):
    return answers[0::3]
Answer = Extract(answers)
Answer

['males are more likely to maintain their video game playing into adulthood',
 'social stigma',
 'cultural differences exist with regard to player preferences for video game activities',
 'hostile and overbearing online behavior',
 'females',
 'video games',
 'western and eastern contexts',
 'the domain',
 'therapeutically',
 'stealth assessment and ecd',
 'an integral part of american culture',
 'video games',
 'hay playing video games',
 'iating pain meeting basic needs and facilitating social connection',
 'facilitating social connection',
 'common game mechanics',
 'creativity',
 'pong',
 'less work has been done',
 'clear learning effects of video game play on variety of cognitive and social skills',
 'evidence',
 'aggression',
 'teach creativity',
 'pong',
 'our world',
 'video games',
 'force in our world and the future ahead',
 'higinbotham',
 'higinbotham',
 'creativity',
 'culture',
 'there are several challenges that the community of creativity researchers need to address',


In [54]:
import pandas as pd
#questions,answer in data frame.
df = pd.DataFrame(quest, columns=['Question'])
df['Answer'] = Answer
df.to_csv('HaystackQA.csv',index=False)
df

,Question,Answer
0,What are the reasons for the lack of female participation in video games?,males are more likely to maintain their video game playing into adulthood
1,What is attached to female video game players the hostile and overbearing on...,social stigma
2,What are games designed by males for?,cultural differences exist with regard to player preferences for video game ...
3,What is the social stigma attached to female video game players?,hostile and overbearing online behavior
4,What is not often socially rewarded for playing video games?,females
...,...,...
173,What does almost every american child can attest their fascination has not w...,hypermuscular characters
174,What kind of benefits do games confer?,cognitive and perceptual
175,What has the video game playing market diversified following?,online multiuser video games
176,What type of video games differ significantly between western and eastern co...,role playing video games
